# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

    '''
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    '''
    def scroll_down(self):
    """A method for scrolling the page."""

        # Get scroll height.
        last_height = self.driver.execute_script("return document.body.scrollHeight")

        while True:

            # Scroll down to the bottom.
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load the page.
            time.sleep(2)

            # Calculate new scroll height and compare with last scroll height.
            new_height = self.driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:

                break

            last_height = new_height
    '''

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 低息貸款吸購屋族！　建案暴增…背後卻藏「缺工」問題

▲近年來小宅建案暴增。（示意圖／記者許宥孺攝）
記者許宥孺／高雄報導
近年來房產市場趨向「小宅化」，尤其以兩房小坪數樓房深獲年輕人喜愛，加上貸款低利率、零自備款等誘因，購屋成交數比往年還多，因此市場出現大量小宅建案，但有資深房仲業者卻指出，大量施工建案可能造成「缺工」問題。
不願具名的資深房仲業者指出，2019年高雄建案量約增加八成，台南也多了三倍，主要原因為貸款低利息、零自備款是吸引購屋族的主因，也因此買房人數有趨多的現象。
▲近年來小宅建案暴增。（示意圖／記者許宥孺攝）
業者表示：「有很多人的心態是，與其把錢放在銀行，不如買房出租，尤其以2房約500、600萬的新房為購屋首選；也有人認為，與其租房不如買房，因此購屋族反而有變多的現象。」
買房人數變多，建商也爆量建案，造成建築產業大缺工，許多建案都無法如期完工。業者指出：「也許預售屋能夠順利賣完，但不見得能賺錢，因為無法如期交屋就拿不到尾款，因此缺工也是營建業未來一大考驗」。
▲近年來小宅建案暴增，背後隱藏缺工問題。（示意圖／記者許宥孺攝）
如何解決缺工問題？業者坦言，光是找到「起厝師傅」就很困難，現代年輕人沒有人願意當建築學徒，產業將會出現斷層。那如何保證建案如期完工？業者表示，以前建商會統一發包工程，現在大多是「發小包」，板模、木工、油漆、光電都是分開派工，建商成本拉高後，就會反應在房屋價格上。
►我沒有遲到，是時間遲到了！
----------------------------------------------------------------------

[1] 快訊／大樂透初二獎號出爐！79人抱走百萬大紅包：幸運兒是你嗎？

▲大樂透連開11天，還加碼360組春節大紅包，9中6即有機會抱走100萬元。（圖／記者柯沛辰攝）
記者柯沛辰／台北報導
大樂透初一頭獎2.4億獎落台東，初二頭獎仍有1億元。春節大紅包9中6的部分，至今已送出91組，其中79組一注獨得100萬元，9組2注均分各得50萬，1組三注均分，1組四注均分，1組五注均分。明晚初三27日將繼續加開9個獎號，對中任6個號碼就有機會抱走百萬